In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from scipy.stats import multivariate_normal

import sys
sys.path.append('C:/Users/matsz/programowanie/Optymalizacja portfela/esg')
import preproccessing as prep
from regime_switching_brownian_motion import RegimeSwitching
from independent_lognormal_model import IndependentLogNormal
import evaluation as eval


### TODO
refactor 

In [2]:
""" spec = [
    ('initial_value', numba.float64[:]),
    ('inflows', numba.float64[:]),
    ('returns', numba.float64[:,:,:]),
    ('capital', numba.float64[:])
]
@jitclass(spec)  """
class Simulation():
    def __init__(self, initial_value, inflows, returns) -> None:
        self.initial_value = np.full(returns.shape[1], initial_value)
        self.returns = returns
        self.inflows = inflows

    def step(self, current_value,inflow,weights,returns):
        portfolio_return = returns @ weights
        next_assets_value =  (current_value + inflow) * (1+portfolio_return)
        return next_assets_value

    def run(self, strategy):
        self.capital = self.initial_value
        T = len(strategy)
            
        for t in range(T):
            self.capital = self.step(self.capital,self.inflows[t],strategy[t],self.returns[t])


class MaxPercentile():

    def __init__(self, percentile) -> None:
        self.percentile = percentile

    def __call__(self, values: np.array) -> float:
        #return np.round(np.percentile(values,self.percentile))
        return np.percentile(values,self.percentile)

class Fitness:
    def __init__(self,simulator: Simulation,portfolios,objective) -> None:
        self.simulator = simulator
        self.portfolios = portfolios
        self.objective = objective

    def __call__(self, solution: np.array) :
        strategy = np.take(self.portfolios,solution, axis=0)
        self.simulator.run(strategy)
        result = self.objective(self.simulator.capital)
        return result
    
def glide_path_grid_search(T, capital, inflows,risk, portfolios, returns):
    l = np.arange(T,0,-1)
    capital = np.repeat(capital,returns.shape[1])

    glide_paths = np.full(T+1,len(portfolios))
    perc = np.zeros((l.shape[0],len(portfolios)))

    objective = MaxPercentile(risk)
    simulator = Simulation(capital,inflows,returns)
    fitness = Fitness(simulator,portfolios,objective)

    for t in l:

        for portfolio in range(len(portfolios)):

            solution = np.full(t,portfolio)
            
            perc[t-1,portfolio] = fitness(solution)
        glide_paths[t-1] = np.argmax(perc[t-1])
    
    return np.flip(glide_paths)[1:]

#optimize glide with resamping
def resampled_glide_path_grid_search(T,capital, inflows, returns, risk, portfolios, number_of_runs):
    batches = np.asarray(np.array_split(returns,number_of_runs,axis=1))
    glides = np.zeros((number_of_runs,T))


    for batch in range(len(batches)):
        glide = glide_path_grid_search(T,capital,inflows,risk,portfolios,batches[batch])
        glides[batch] = glide

    glides = np.int32(glides)

    result = np.zeros(T)
    for t in range(T):
        result[t] = np.argmax(np.bincount(glides[:,t], minlength=11))
        
    return pd.Series(result, index=range(T,0,-1))

def pct_change(a, freq=1):
    filtered = a[::freq]
    return filtered[1:]/filtered[:-1] -1

In [3]:
def expected_returns(period):
    return pd.Series({
		'ACWI': 0.07/period,
		'^TBSP': 0.035/period,
		'XAUPLN': 0.05/period,
		'MWIG40': 0.065/period,
		'IHYU.L': 0.038/period,
		'PLOPLN3M': 0.03/period,
		'edo': 0.025/period
})
#acwi and edo
corr = np.array([[1,0],[0,1]])
std = np.array([0.135,0.02])
cov_matrix = prep.corr_to_cov(corr, std)

assets = ['ACWI', 'edo']
means = expected_returns(1)[assets]
number_of_scenarios = 10 * 10000
T = 50
sim_returns = []

for s in range (number_of_scenarios):
    sim_returns.append(multivariate_normal.rvs(means,cov_matrix, T))

#calculate inflation
sim_returns = np.expm1(sim_returns)
sim_returns[:,:,1] = np.where(sim_returns[:,:,1]<0,0,sim_returns[:,:,1])
sim_returns[:,:,1] = sim_returns[:,:,1] + 0.0123

sim_returns = np.array(sim_returns).transpose(1,0,2)
sim_returns.shape

(50, 100000, 2)

In [3]:
number_of_scenarios = 50 * 2000

In [13]:
model = RegimeSwitching(models = [IndependentLogNormal(mu=0.0127,sigma=0.027),
              IndependentLogNormal(mu=-0.0136, sigma=0.0510)],
    probs = [[0.958,0.042],[0.108, 0.892]]
)
equity = model.scenarios(100, current_regimes=0, dt=1,n_scenarios=number_of_scenarios,n_steps=600)
equity = equity.T
equity_returns = pct_change(equity,12)

equity_returns.shape

(50, 100000)

In [14]:
inflation = np.random.normal(loc=0.025, scale=0.015,size=(50,number_of_scenarios))
inflation = pd.DataFrame(data=inflation)
sim_inflation = np.expm1(inflation)

sim_inflation = np.where(sim_inflation < 0,0,sim_inflation)
sim_inflation = sim_inflation + 0.0123


In [15]:
sim_returns = np.stack([equity_returns,sim_inflation],axis=2)
sim_returns.shape

(50, 100000, 2)

In [4]:
""" portfolios = [[0.0,1.0],
             [0.2,0.8],
             [0.4,0.6],
             [0.6,0.4],
             [0.8,0.2],
             [1.0,0.0]]  """

portfolios = [[0.0,1.0],
             [0.1,0.9],
             [0.2,0.8],
             [0.3,0.7],
             [0.4,0.6],
             [0.5,0.5],
             [0.6,0.4],
             [0.7,0.3],
             [0.8,0.2],
             [0.9,0.1],
             [1.0,0.0]]



In [28]:
number_of_runs = 10
T = 50
capital = 10.0
#risks = [5,15,25,35,45]
#risks = [10,20,30,40,50]
risks = [5,10,15,20,25,30,35,40,45]
inflows = np.cumproduct(np.full(T,1.025))*5

glide_paths = np.zeros((T,len(risks)))

for risk in range(len(risks)):
    glide_paths[:,risk] = resampled_glide_path_grid_search(T,capital,inflows,sim_returns,risks[risk],portfolios,number_of_runs)

glide_paths = pd.DataFrame(data=np.int32(glide_paths),index=range(T,0,-1), columns=risks)
glide_paths

,5,10,15,20,25,30,35,40,45
50,5,10,10,10,10,10,10,10,10
49,5,9,10,10,10,10,10,10,10
48,5,9,10,10,10,10,10,10,10
47,5,9,10,10,10,10,10,10,10
46,4,9,10,10,10,10,10,10,10
45,4,9,10,10,10,10,10,10,10
44,4,8,10,10,10,10,10,10,10
43,4,8,10,10,10,10,10,10,10
42,4,8,10,10,10,10,10,10,10
41,4,8,10,10,10,10,10,10,10


In [12]:
glide_paths.to_excel('glide_paths_norm_5.11.2023.xlsx')

In [18]:
#TODO sim returns for glide paths

inflows = np.cumproduct(np.full(T,1.025))*0
initial_capital = 1.0
simulator = Simulation(initial_capital,inflows,sim_returns)

glide_returns = np.zeros((T,len(risks)))

for t in range(0,T,):
    for risk in range(len(risks)):
        glide_path = glide_paths.loc[t+1:,risks[risk]]
        fitness = Fitness(simulator,portfolios,MaxPercentile(risks[risk])) 
        glide_returns[t,risk] = fitness(glide_path)         
      
glide_returns = pd.DataFrame(data=np.flip(glide_returns,axis=0),index=range(T,0,-1),columns=risks)
glide_returns

,5,10,15,20,25,30,35,40,45
50,5.932297,6.451760,6.966044,8.002368,9.894097,13.236661,16.510156,19.417197,22.854631
49,5.714694,6.207383,6.685165,7.623463,9.376465,12.533500,15.562053,18.365878,21.476865
48,5.499550,5.966931,6.415300,7.257627,8.883922,11.799204,14.624873,17.218938,20.108063
47,5.292402,5.744502,6.157734,6.922433,8.421650,11.156253,13.749740,16.146332,18.856288
46,5.093609,5.522366,5.883380,6.609172,7.998593,10.515316,12.937210,15.220875,17.739896
45,4.905281,5.287329,5.641027,6.303925,7.602123,9.947700,12.209612,14.306896,16.681720
44,4.716556,5.083179,5.417472,6.021186,7.225623,9.408888,11.495951,13.453544,15.617453
43,4.540740,4.884793,5.204682,5.764085,6.862173,8.910293,10.836529,12.660914,14.693993
42,4.371593,4.698580,4.999320,5.501562,6.519652,8.388528,10.209309,11.943934,13.750955
41,4.209938,4.515826,4.799027,5.237017,6.198169,7.914036,9.650660,11.216412,12.963678


In [20]:
description = pd.DataFrame(data={'Opis': """"
Numer_of_scenarios = 250000,
number_of_runs = 100
T = 50
capital = 10.0
risks = [5,10,15,20,25,30,35,40,45]
inflows = np.cumproduct(np.full(T,1.025))*5
"""},index=[1])

In [21]:
with pd.ExcelWriter('glide_paths_norm_17.12.2023.xlsx', engine='openpyxl', mode='w') as writer:  
    glide_paths.to_excel(writer, sheet_name='Glide_paths')
    glide_returns.to_excel(writer, sheet_name='Returns')
    #description.to_excel(writer,sheet_name='Description')


In [50]:
print(105/100-1)
print(109/105-1)
print(107/100-1)
print(106/107-1)

0.050000000000000044
0.03809523809523818
0.07000000000000006
-0.009345794392523366


In [49]:
arr = np.array([100,103,107,105,99,103,109,111])
arr2 = np.array([[100,103,107,105,99,103,109,111],[100,104,109,107,99,102,106,110]]).T
def pct_change(a, freq=1):
    filtered = a[::freq]
    return filtered[1:]/filtered[:-1] -1

pct_change(arr2,3)


array([[ 0.05      ,  0.07      ],
       [ 0.03809524, -0.00934579]])

In [5]:
def glide_path_grid_search(T, capital, inflows,risk, portfolios, returns):
    l = np.arange(T,0,-1)
    capital = np.repeat(capital,returns.shape[1])

    glide_paths = np.full(T+1,len(portfolios))
    perc = np.zeros((l.shape[0],len(portfolios)))

    objective = MaxPercentile(risk)
    simulator = Simulation(capital,inflows,returns)
    fitness = Fitness(simulator,portfolios,objective)

    for t in l:

        for portfolio in range(len(portfolios)):

            solution = np.full(t,portfolio)
            
            perc[t-1,portfolio] = fitness(solution)
        glide_paths[t-1] = np.argmax(perc[t-1])
    
    return np.flip(glide_paths)[1:]

In [75]:
T = 50
capital = 10.0
#risks = [5,15,25,35,45]
#risks = [10,20,30,40,50]
risks = [5,10,15,20,25,30,35,40,45]
inflows = np.cumproduct(np.full(T,1.025))*5
simulator = Simulation(capital,inflows,sim_returns)
glide = np.zeros((T,len(risks)),dtype=np.int32)

for risk in range(len(risks)):
    objective = MaxPercentile(risks[risk])
    fitness = Fitness(simulator,portfolios,objective)
    
    perc = np.zeros((T,len(portfolios)))
    for t in range(T):
        for portfolio in range(len(portfolios)):
            solution = []
            solution = glide[risk,0:t+1]
            solution[-1] = portfolio
            perc[t,portfolio] = fitness(solution)
        glide[t,risk] = np.argmax(perc[t])

glide = pd.DataFrame(data=glide,index=range(T,0,-1), columns=risks)
glide

,5,10,15,20,25,30,35,40,45
50,0,1,1,1,1,1,1,3,10
49,10,2,2,3,3,4,6,10,10
48,10,10,3,3,4,5,8,10,10
47,10,10,10,4,5,6,9,10,10
46,10,10,10,10,6,6,8,10,10
45,10,10,10,10,10,7,9,10,10
44,10,10,10,10,10,10,10,10,10
43,10,10,10,10,10,10,10,10,10
42,10,10,10,10,10,10,10,10,10
41,3,4,5,6,7,8,10,10,10


In [46]:
objective = MaxPercentile(30)
simulator = Simulation(capital,inflows,sim_returns)
fitness = Fitness(simulator,portfolios,objective)

In [68]:
fitness(glide_paths.loc[25:0,25])

352.18949983144057

In [67]:
fitness(glide.iloc[0:25,0])

360.37900992644694

In [87]:
from scipy.stats import norm

1-norm.cdf(0.05,0.07,0.15)

0.553035116623614